# Reinforcement Learning: Min Distance RL with CARLA CL (Larger Model) 13/01/22

At the recommednation of one of the authors of the paper I will try to adapt the following [tutorial](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) to work acheive the same results as in the paper.

Notes: Pygame only runs on python 3.7, pytorch must be install direclty into the env

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

import carla
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image

from torch.utils.tensorboard import SummaryWriter

import gym
import gym_carla

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# parameters for the gym_carla environment
params = {
    'number_of_vehicles': 0,
    'number_of_walkers': 0,
    'display_size': 256,  # screen size of bird-eye render
    'max_past_step': 1,  # the number of past steps to draw
    'dt': 0.1,  # time interval between two frames
    'discrete': True,  # whether to use discrete control space
    'discrete_acc': [2.0],  # discrete value of accelerations
    'discrete_steer': [-0.3, 0.0, 0.3],  # discrete value of steering angles
    'continuous_accel_range': [-3.0, 3.0],  # continuous acceleration range
    'continuous_steer_range': [-0.3, 0.3],  # continuous steering angle range
    'ego_vehicle_filter': 'vehicle.lincoln*',  # filter for defining ego vehicle
    'port': 2000,  # connection port
    'town': 'Town04',  # which town to simulate
    'task_mode': 'random',  # mode of the task, [random, roundabout (only for Town03)]
    'max_time_episode': 500,  # maximum timesteps per episode
    'max_waypt': 12,  # maximum number of waypoints
    'obs_range': 32,  # observation range (meter)
    'lidar_bin': 0.125,  # bin size of lidar sensor (meter)
    'd_behind': 12,  # distance behind the ego vehicle (meter)
    'out_lane_thres': 1.8,  # threshold for out of lane
    'desired_speed': 3,  # desired speed (m/s)
    'max_ego_spawn_times': 20,  # maximum times to spawn ego vehicle
    'display_route': True,  # whether to render the desired route
    'pixor_size': 64,  # size of the pixor labels
    'pixor': False,  # whether to output PIXOR observation
    'use_fixed':'E',
    'weather':'ClearNoon'
}

# Set gym-carla environment
env = gym.make('carla-v0', params=params)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
connecting to Carla server...
Carla server connected!
WeatherParameters(cloudiness=5.000000, cloudiness=5.000000, precipitation=0.000000, precipitation_deposits=0.000000, wind_intensity=10.000000, sun_azimuth_angle=-1.000000, sun_altitude_angle=45.000000, fog_density=2.000000, fog_distance=0.750000, fog_falloff=0.100000, wetness=0.000000, scattering_intensity=1.000000, mie_scattering_scale=0.030000, rayleigh_scattering_scale=0.033100)
372


In [3]:

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    
    def __len__(self):
        return len(self.memory)


In [12]:
class DQN(nn.Module):

    def __init__(self, outputs):
        super(DQN, self).__init__()
        
        self.lin1 = nn.Linear(9,80)
        self.lin2 = nn.Linear(80,50)
        self.lin3 = nn.Linear(50,25)
        self.lin4 = nn.Linear(25,15)
        self.lin5 = nn.Linear(15,8)
        self.lin6 = nn.Linear(8,3)

    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        x = F.relu(self.lin4(x))
        x = F.relu(self.lin5(x))
        x = self.lin6(x)
        
        return x.view(x.size(0), -1)

In [13]:
BATCH_SIZE = 512
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200000
TARGET_UPDATE = 256

writer = SummaryWriter()

n_actions = env.action_space.n

policy_net = DQN(n_actions).to(device)
policy_net.load_state_dict(torch.load('./model_params_CL/model_8_E.best'))
target_net = DQN(n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

# Model <= 7
optimizer = optim.RMSprop(policy_net.parameters(),lr=0.005)

#Model 8 & from 2022 
#optimizer = optim.Adam(policy_net.parameters(),lr=0.005)


memory = ReplayMemory(7500)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            #arg max select the idex of the largest value and view changes shape from (1,) to (1,1)
            #try test net
            return policy_net(state.float()).argmax().view(1,1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)


episode_durations = []

In [14]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    #reshape state_batch for nn
  
    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    
    # selects column of output that was selceted 
    state_action_values = policy_net(torch.reshape(state_batch,(BATCH_SIZE,1,9)).float()).gather(1,action_batch)
    
    
    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(torch.reshape(non_final_next_states,(list(non_final_next_states.shape)[0]//9,1,9)).float()).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp(-1, 1)
    optimizer.step()
    
    return loss

In [ ]:
num_episodes = 600
use_fixed_idx = 0
env.use_fixed = 'M'
levels = ['E','M','H']
next_lvl = 300
min_overall_loss = 1000
for i_episode in range(num_episodes):
    rewards = []
    # Initialize the environment and state
    obs = env.reset()
    #ego_dir retirves the distance and angle from vehicle to nearest waypoint
    ego_location = env.ego.get_location()
    ego_dir = gym_carla.envs.misc.get_lane_dis(env.waypoints,ego_location.x,ego_location.y)
    #pos gets a distanc d and array w which has to be seperated out in below line
    ego_pos = np.asarray((ego_dir[0],ego_dir[1][0],ego_dir[1][1]),dtype=np.float32)
    state = np.concatenate((ego_pos,np.zeros(6)))
    state = torch.tensor(state)
    # Resize, and add a batch dimension (BCHW)
    loss = episode_loss = 1000
    for t in count():
        # Select and perform an action
        action = select_action(state)
        _ , reward, done, info  = env.step(action.item())
        rewards.append(reward)
        reward = torch.tensor([reward], device=device)

        if not done:
            
            #pos gets a distanc d and array w which has to be seperated out in below line
            pos = np.asarray((info['position'][0],info['position'][1][0],info['position'][1][1]))
            ang = np.asarray(info['angular_vel'])
            acc = np.asarray(info['acceleration'])
            steer = np.asarray(info['steer'])
            next_state = np.concatenate((pos, ang, acc, steer), axis=None)
            next_state = torch.tensor(next_state)
        else:
            next_state = None

        memory.push(state, action, next_state, reward)
        
        state = next_state

        # Perform one step of the optimization (on the policy network)
        loss = optimize_model()
        
        #if loss is better than previous minimum save mode
        if loss is not None and loss < episode_loss:
            episode_loss = loss
            writer.add_scalar("Loss/train", loss, i_episode)
        
        if done:
            episode_durations.append(t + 1)
            break
            
    # Update the target network
    if steps_done % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
    len_episode = t+1
    writer.add_scalar("Lenght/Epoch", len_episode, i_episode)
    
    if i_episode > 0 and i_episode % next_lvl == 0:
        #saving model after each level
        torch.save(target_net.state_dict(), './model_params_CL/model_10'+levels[use_fixed_idx]+'.final')
        print(levels[use_fixed_idx])
        use_fixed_idx += 1
        env.use_fixed = levels[use_fixed_idx]
    
    #save model if better than previous episode
    if episode_loss < min_overall_loss:
        min_overall_loss = episode_loss
        torch.save(target_net.state_dict(), './model_params_CL/model_10.best')
    

print('Complete')

writer.flush()
writer.close()
env.close()
plt.ioff()
plt.show()

### Visualise Models

In [6]:
vis_net = DQN(n_actions).to(device)
vis_net.load_state_dict(torch.load('./model_params_CL/model_9E.final'))
vis_net.eval()



DQN(
  (lin1): Linear(in_features=9, out_features=80, bias=True)
  (lin2): Linear(in_features=80, out_features=50, bias=True)
  (lin3): Linear(in_features=50, out_features=25, bias=True)
  (lin4): Linear(in_features=25, out_features=15, bias=True)
  (lin5): Linear(in_features=15, out_features=8, bias=True)
  (lin6): Linear(in_features=8, out_features=3, bias=True)
)

In [11]:
num_episodes = 8
env.use_fixed = 'H'
use_fixed_idx = 0
levels = ['E','M','H']
next_lvl = 4
min_overall_loss = 1000
for i_episode in range(num_episodes):
    rewards = 0
    # Initialize the environment and state
    obs = env.reset()
    #ego_dir retirves the distance and angle from vehicle to nearest waypoint
    ego_location = env.ego.get_location()
    ego_dir = gym_carla.envs.misc.get_lane_dis(env.waypoints,ego_location.x,ego_location.y)
    #pos gets a distanc d and array w which has to be seperated out in below line
    ego_pos = np.asarray((ego_dir[0],ego_dir[1][0],ego_dir[1][1]),dtype=np.float32)
    state = np.concatenate((ego_pos,np.zeros(6)))
    state = torch.tensor(state)
    # Resize, and add a batch dimension (BCHW)
    loss = episode_loss = 1000
    for t in count():
        # Select and perform an actionii#
        with torch.no_grad():
            print(state.float())
            action = vis_net(state.float()).argmax().view(1,1)
            print(vis_net(state.float()))
            print(action)
        _ , reward, done, info  = env.step(action.item())
        rewards += reward
        
        #pos gets a distanc d and array w which has to be seperated out in below line
        pos = np.asarray((info['position'][0],info['position'][1][0],info['position'][1][1]))
        ang = np.asarray(info['angular_vel'])
        acc = np.asarray(info['acceleration'])
        steer = np.asarray(info['steer'])
        next_state = np.concatenate((pos, ang, acc, steer), axis=None)
        #update state
        state = torch.tensor(next_state)

        if done:
            print('########')
            episode_durations.append(t + 1)
            break


tensor([-3.1056e-05, -8.9173e-03,  9.9996e-01,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00])
tensor([[-0.2032],
        [ 0.0103],
        [ 0.2773]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([-3.1056e-05, -8.9173e-03,  9.9996e-01,  1.7825e-06,  1.2543e-06,
         5.4343e-11,  8.0013e-07, -4.3116e-07, -3.0000e-01])
tensor([[-0.2043],
        [ 0.0119],
        [ 0.2770]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([-3.1056e-05, -8.9173e-03,  9.9996e-01, -1.0637e-05,  3.9014e-05,
        -2.1216e-09,  6.2926e-07,  4.8295e-07, -3.0000e-01])
tensor([[-0.2043],
        [ 0.0119],
        [ 0.2770]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([-3.1056e-05, -8.9173e-03,  9.9996e-01,  3.6844e-06,  2.9460e-06,
        -6.8809e-09, -4.8758e-07, -5.5761e-07, -3.0000e-01])
tensor([[-0.2043],
        [ 0.0119],
        [ 0.2770]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([-3.1056e-05, -8.9173e-03,  9.999

########
tensor([0.0035, 1.0000, 0.0057, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])
tensor([[-0.2012],
        [ 0.0077],
        [ 0.2780]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 3.4923e-03,  9.9998e-01,  5.6919e-03, -9.5917e-06, -5.3534e-06,
        -6.6931e-09, -4.1164e-08,  9.2929e-07, -3.0000e-01])
tensor([[-0.2022],
        [ 0.0087],
        [ 0.2776]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 3.4923e-03,  9.9998e-01,  5.6919e-03, -4.9055e-05,  7.0062e-06,
        -4.2915e-09,  3.8055e-08, -1.2331e-06, -3.0000e-01])
tensor([[-0.2022],
        [ 0.0087],
        [ 0.2776]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 3.4923e-03,  9.9998e-01,  5.6919e-03,  8.6982e-06, -5.8808e-06,
         2.6663e-08, -2.1080e-07, -5.3715e-07, -3.0000e-01])
tensor([[-0.2022],
        [ 0.0087],
        [ 0.2776]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 3.4923e-03,  9.9998e-01,  5.6919e-03, -5.9295e-06, -5.2420e-06,
         4.

########
tensor([-0.0948, -0.0104,  0.9999,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000])
tensor([[-0.2035],
        [ 0.0106],
        [ 0.2773]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([-9.4813e-02, -1.0447e-02,  9.9995e-01, -2.1151e-06, -4.3436e-05,
         1.3859e-08,  5.9593e-07,  1.4894e-07, -3.0000e-01])
tensor([[-0.2045],
        [ 0.0122],
        [ 0.2768]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([-9.4813e-02, -1.0447e-02,  9.9995e-01, -1.2928e-06, -5.0356e-05,
        -3.2000e-08,  1.5669e-06, -2.6038e-07, -3.0000e-01])
tensor([[-0.2045],
        [ 0.0122],
        [ 0.2768]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([-9.4813e-02, -1.0447e-02,  9.9995e-01,  1.1697e-05,  1.5018e-05,
        -1.2742e-08,  3.1824e-07, -1.4323e-07, -3.0000e-01])
tensor([[-0.2045],
        [ 0.0122],
        [ 0.2768]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([-9.4813e-02, -1.0447e-02,  9.9995e-01, -1.1709e-06, -4.4262

tensor([ 1.5548e+00, -1.0447e-02,  9.9995e-01,  1.1209e+00, -1.4670e+00,
        -1.6134e+01,  3.5933e+00,  3.8839e+00, -3.0000e-01])
tensor([[-0.2469],
        [-0.0385],
        [ 0.2716]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 1.7348e+00, -1.0447e-02,  9.9995e-01,  9.6517e-01, -1.6644e+00,
        -1.9977e+01,  4.7494e+00,  4.6424e+00, -3.0000e-01])
tensor([[-0.2636],
        [-0.0511],
        [ 0.2731]], device='cuda:0')
tensor([[2]], device='cuda:0')
########
tensor([-1.0034e-06,  1.3464e-02, -9.9991e-01,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00])
tensor([[-0.2058],
        [ 0.0073],
        [ 0.2742]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([-1.0034e-06,  1.3464e-02, -9.9991e-01,  2.9152e-06,  7.5984e-07,
         5.4642e-09, -1.5289e-07, -1.8350e-08, -3.0000e-01])
tensor([[-0.2062],
        [ 0.0077],
        [ 0.2742]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([-1.0034e-06,  1.3464e-0

tensor([ 1.7908e+00,  1.3464e-02, -9.9991e-01,  1.2468e+00, -1.0534e+00,
        -2.0071e+01,  9.3595e-01,  3.2775e+00, -3.0000e-01])
tensor([[-0.2747],
        [-0.0524],
        [ 0.2770]], device='cuda:0')
tensor([[2]], device='cuda:0')
########
tensor([1.5068e-04, 9.9998e-01, 5.6919e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00])
tensor([[-0.2012],
        [ 0.0077],
        [ 0.2780]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 1.5068e-04,  9.9998e-01,  5.6919e-03,  1.0384e-05,  8.8917e-06,
         7.6524e-09,  6.6770e-08,  1.5969e-07, -3.0000e-01])
tensor([[-0.2022],
        [ 0.0087],
        [ 0.2775]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 1.5068e-04,  9.9998e-01,  5.6919e-03, -3.2528e-05, -6.3355e-06,
        -5.6827e-08, -6.3414e-07,  1.7609e-07, -3.0000e-01])
tensor([[-0.2022],
        [ 0.0087],
        [ 0.2775]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 1.5068e-04,  9.9998e-01,  5.691

tensor([ 1.5404e+00,  9.9998e-01,  5.6919e-03, -8.9426e-01, -8.0830e-01,
        -1.7024e+01,  1.9619e+00, -2.1808e+00, -3.0000e-01])
tensor([[-0.2552],
        [-0.0448],
        [ 0.2727]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 1.7082e+00,  9.9998e-01,  5.6919e-03, -1.2748e+00, -8.7443e-01,
        -1.9319e+01,  3.1372e+00, -3.4099e+00, -3.0000e-01])
tensor([[-0.2650],
        [-0.0504],
        [ 0.2803]], device='cuda:0')
tensor([[2]], device='cuda:0')
########
tensor([-2.8200e-05, -3.0849e-03,  1.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00])
tensor([[-0.2032],
        [ 0.0103],
        [ 0.2773]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([-2.8200e-05, -3.0849e-03,  1.0000e+00, -2.4399e-06, -2.4430e-05,
        -3.4619e-08, -2.0564e-06,  4.2140e-07, -3.0000e-01])
tensor([[-0.2043],
        [ 0.0118],
        [ 0.2770]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([-2.8200e-05, -3.0849e-0

tensor([ 1.5131e+00, -3.0849e-03,  1.0000e+00,  6.5020e-01, -5.9462e-01,
        -1.5328e+01,  1.3526e+00,  1.0070e+00, -3.0000e-01])
tensor([[-0.2471],
        [-0.0435],
        [ 0.2651]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 1.6559e+00, -3.0849e-03,  1.0000e+00,  7.0173e-01, -8.5847e-01,
        -1.6919e+01,  2.2400e+00,  1.9344e+00, -3.0000e-01])
tensor([[-0.2514],
        [-0.0454],
        [ 0.2661]], device='cuda:0')
tensor([[2]], device='cuda:0')
########
tensor([-3.1056e-05, -8.9173e-03,  9.9996e-01,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00])
tensor([[-0.2032],
        [ 0.0103],
        [ 0.2773]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([-3.1056e-05, -8.9173e-03,  9.9996e-01,  1.7825e-06,  1.2543e-06,
         5.4343e-11,  8.0013e-07, -4.3116e-07, -3.0000e-01])
tensor([[-0.2043],
        [ 0.0119],
        [ 0.2770]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([-3.1056e-05, -8.9173e-0

KeyboardInterrupt: 